In [2]:
import os
import sys

# ==========================================
# Environment Setup: Automatic Detection
# ==========================================
try:
    # Check if running in Google Colab
    from google.colab import drive
    print("🚀 Running in Google Colab")

    # 1. Mount Google Drive
    drive.mount('/content/drive')

    # 2. Change working directory to the project folder in Google Drive
    colab_path = '/content/drive/MyDrive/データサイエンス個人学習/Football_Tactical_Analysis'

    if os.path.exists(colab_path):
        os.chdir(colab_path)
        print(f"📂 Current Directory moved to: {os.getcwd()}")
    else:
        print(f"⚠️ Path not found: {colab_path}")

except ImportError:
    # Running Locally
    print("💻 Running Locally")

    # Identify the project root directory regardless of the notebook location.
    # If the current directory is 'notebooks', move up one level to the project root.
    if os.path.basename(os.getcwd()) == "notebooks":
        os.chdir("..")
        print("⬆️ Moved up from 'notebooks' folder to project root.")

    print(f"📂 Current Directory: {os.getcwd()}")

# ==========================================
# Add Project Root to Path & Import Utils
# ==========================================
# Ensure the root directory is in sys.path to find EN_soccer_utils.py
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

import EN_soccer_utils as utils
print("✅ Successfully imported EN_soccer_utils")

🚀 Running in Google Colab
Mounted at /content/drive
📂 Current Directory moved to: /content/drive/MyDrive/データサイエンス個人学習/Football_Tactical_Analysis
✅ Successfully imported EN_soccer_utils


In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import Image, display

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import log_loss
import datetime as dt
import os
import pickle
from sklearn.calibration import calibration_curve
import time
import glob


# Display settings
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)



"""{
  "type": {"id": 16, "name": "Shot"},  
  "player": {"id": 123, "name": "Messi"},  
  "location": [102.3, 40.5]  
}

Acquire the JSON file in the format above and convert it to the format below using json_normalize:  
  
type_id        = 16  
type_name      = "Shot"  
player_id      = 123  
player_name    = "Messi"  
location       = [102.3, 40.5]  

pd.json_normalize is used to convert nested JSON (like StatsBomb)
into a flat, analyzable DataFrame.

In [ ]:
# Acquire data for 380 matches of the Premier League 2015-2016 season & extract and combine only shot data

# Save path
# NOTE: I translated the folder names below. Please rename your Drive folders or update this path.
SAVE_PATH = "data/all_league_shots_2015_16_with_assist.csv"

if os.path.exists(SAVE_PATH):
    print(f"📂 Loading data from CSV: {SAVE_PATH}")
    all_league_shots_df = pd.read_csv(SAVE_PATH)

else:
    print("🌐 Acquiring data from the Web")

    # Premier League 2015/2016
    COMPETITION_ID = 2
    SEASON_ID = 27

    url = f"https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/{COMPETITION_ID}/{SEASON_ID}.json"
    matches = requests.get(url).json()
    all_matches_df = pd.json_normalize(matches)

    all_shots_list = []
    total_matches = len(all_matches_df)

    print(f"🚀 Analysis Start: Total {total_matches} matches")

    for i, match_id in enumerate(all_matches_df["match_id"]):
        if (i + 1) % 10 == 0 or (i + 1) == total_matches:
            print(f"📊 {i + 1}/{total_matches}")

        url_events = f"https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/{match_id}.json"
        events_resp = requests.get(url_events)

        # Error handling (because retrieval sometimes fails)
        if events_resp.status_code != 200:
            print(f"⚠️ Match {match_id} skipped (Error {events_resp.status_code})")
            continue

        events = events_resp.json()
        df = pd.json_normalize(events, sep="_")

        # Check if necessary columns exist
        if "type_name" in df.columns:
            # ----------------------------------------------------
            # Extraction of shot data for one match
            # ----------------------------------------------------
            shots = df[df["type_name"] == "Shot"].copy()

            if shots.empty:
                continue

            # ----------------------------------------------------
            # Preparation of pass information for one match (for assist identification)
            # ----------------------------------------------------
            # Extract only pass events and set ID as index (for faster search)
            pass_events = df[df["type_name"] == "Pass"].set_index("id")

            # Function to get assist info (executed on this match's df)
            # shot_key_pass_id holds the id of the pass that led to the shot.
            def get_assist_info(row):
                # If shot_key_pass_id exists (if there is an assist)
                if "shot_key_pass_id" in row and pd.notnull(row["shot_key_pass_id"]):
                    pass_id = row["shot_key_pass_id"]

                    if pass_id in pass_events.index:
                        pass_event = pass_events.loc[pass_id]

                        # Pass starting point (x, y)
                        pass_loc = pass_event.get("location", [np.nan, np.nan])
                        if not isinstance(pass_loc, list): pass_loc = [np.nan, np.nan] # Safety measure

                        # Pass type
                        # Use .get() since the column might not exist in the data
                        is_cross = pass_event.get("pass_cross", False)
                        is_cutback = pass_event.get("pass_cut_back", False)
                        is_through_ball = pass_event.get("pass_through_ball", False)
                        pass_height = pass_event.get("pass_height_name", "Unknown") # Ground, High etc.

                        return pd.Series([pass_loc[0], pass_loc[1], is_cross, is_cutback, is_through_ball, pass_height])

                # If no assist or not found
                return pd.Series([np.nan, np.nan, False, False, False, "None"])

            # Apply function to create new columns
            shots[["assist_x", "assist_y", "is_cross", "is_cutback", "is_through_ball", "pass_height"]] = shots.apply(get_assist_info, axis=1)

            # ----------------------------------------------------
            # 3. Expansion of coordinates (existing process)
            # ----------------------------------------------------
            if "location" in shots.columns:
                shots[["x", "y"]] = shots["location"].apply(
                    lambda loc: pd.Series(loc) if isinstance(loc, list) else pd.Series([np.nan, np.nan])
                )

            if "shot_end_location" in shots.columns:
                shots[["shot_end_x", "shot_end_y"]] = shots["shot_end_location"].apply(
                    lambda loc: pd.Series(loc[:2]) if isinstance(loc, list) else pd.Series([np.nan, np.nan])
                )

            shots["match_id"] = match_id
            all_shots_list.append(shots)

    # Combine
    all_league_shots_df = pd.concat(all_shots_list, ignore_index=True)

    # Delete unnecessary columns (reduce file size)
    drop_cols = [c for c in ["location", "shot_end_location"] if c in all_league_shots_df.columns]
    all_league_shots_df = all_league_shots_df.drop(columns=drop_cols)

    # Fill missing values (True/False types filled with False, numbers can be NaN but filled for easier handling if needed)
    bool_cols = ["is_cross", "is_cutback", "is_through_ball"]
    all_league_shots_df[bool_cols] = all_league_shots_df[bool_cols].fillna(False)

    all_league_shots_df.to_csv(SAVE_PATH, index=False)
    print(f"💾 Saved CSV with assist info: {SAVE_PATH}")

🌐 Acquiring data from the Web
🚀 Analysis Start: Total 380 matches
📊 10/380
📊 20/380
📊 30/380
📊 40/380
📊 50/380
📊 60/380
📊 70/380
📊 80/380
📊 90/380
📊 100/380
📊 110/380
📊 120/380
📊 130/380
📊 140/380
📊 150/380
📊 160/380
📊 170/380
📊 180/380
📊 190/380
📊 200/380
📊 210/380
📊 220/380
📊 230/380
📊 240/380
📊 250/380
📊 260/380
📊 270/380
📊 280/380
📊 290/380
📊 300/380
📊 310/380
📊 320/380
📊 330/380
📊 340/380
📊 350/380
📊 360/380
📊 370/380
📊 380/380


/tmp/ipython-input-188392082.py:112: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_league_shots_df[bool_cols] = all_league_shots_df[bool_cols].fillna(False)


💾 Saved CSV with assist info: data/all_league_shots_2015_16_with_assist.csv
